In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image
import glob

def prepare_classification_dataset(
    source_data_path,
    output_path,
    train_ratio=0.8,
    validate_images=True
):
    """
    Prepares a dataset for YOLOv8 classification training by organizing images into the expected directory structure.
    
    Args:
        source_data_path (str): Path to your raw data
        output_path (str): Where to create the organized dataset
        train_ratio (float): Proportion of data to use for training (default 0.8)
        validate_images (bool): Whether to check if images are valid before copying
    """
    # Create main directories
    train_dir = os.path.join(output_path, 'train')
    val_dir = os.path.join(output_path, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    # Get list of classes (assuming class names are directory names)
    classes = [d for d in os.listdir(source_data_path) 
              if os.path.isdir(os.path.join(source_data_path, d))]

    for class_name in classes:
        # Create class directories in train and val
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

        # Get all images for this class
        class_path = os.path.join(source_data_path, class_name)
        images = []
        for ext in ('*.jpg', '*.jpeg', '*.png', '*.bmp', '*.webp'):
            images.extend(glob.glob(os.path.join(class_path, ext)))

        # Split into train and validation sets
        train_images, val_images = train_test_split(
            images, 
            train_size=train_ratio,
            random_state=42  # For reproducibility
        )

        # Function to safely copy images
        def copy_images(image_list, destination_dir):
            for img_path in image_list:
                if validate_images:
                    try:
                        # Attempt to open the image to verify it's valid
                        with Image.open(img_path) as img:
                            img.verify()
                    except Exception as e:
                        print(f"Skipping corrupted image {img_path}: {str(e)}")
                        continue

                # Copy the image to appropriate directory
                dest_path = os.path.join(destination_dir, class_name, os.path.basename(img_path))
                shutil.copy2(img_path, dest_path)

        # Copy images to train and validation directories
        copy_images(train_images, train_dir)
        copy_images(val_images, val_dir)

    return len(classes), classes

In [3]:
from ultralytics import YOLO

# Step 1: Prepare the dataset
source_path = r"D:\Kananat\TF_TMJOA_jpg_x_5px"
output_path = r"D:\Kananat\TF_TMJOA_jpg_x_5px_yolo"

num_classes, class_names = prepare_classification_dataset(
    source_path,
    output_path,
    train_ratio=0.8,
    validate_images=True
)

In [ ]:
model = YOLO('yolov8m-cls.pt')  # Load the classification model

# Start training
results = model.train(
    data="D:\Kananat\TF_TMJOA_jpg_x_5px_yolo",
    epochs=100,
    imgsz=224,
    batch=32,
    device='0'  # Use '0' for first GPU, 'cpu' for CPU training
)

100%|██████████| 32.7M/32.7M [00:00<00:00, 98.9MB/s]


Ultralytics 8.3.44  Python-3.12.0 torch-2.5.1 CUDA:0 (Quadro P4000, 8192MiB)
engine\trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=C:/Users/kanan/Desktop/Work_at_IGB/Code/datasets, epochs=100, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

100%|██████████| 5.35M/5.35M [00:00<00:00, 67.7MB/s]


AMP: checks passed 


train: Scanning C:\Users\kanan\Desktop\Work_at_IGB\Code\datasets\train... 50442 images, 0 corrupt: 100%|██████████| 50442/50442 [00:22<00:00, 2270.91it/s]


...


      Epoch    GPU_mem       loss  Instances       Size


    100/100      1.82G    0.01157         10        224: 100%|██████████| 1577/1577 [05:26<00:00,  4.83it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 198/198 [00:24<00:00,  7.94it/s]


                   all      0.986          1

100 epochs completed in 9.740 hours.
Optimizer stripped from runs\classify\train12\weights\last.pt, 31.8MB
Optimizer stripped from runs\classify\train12\weights\best.pt, 31.8MB

Validating runs\classify\train12\weights\best.pt...
Ultralytics 8.3.44  Python-3.12.0 torch-2.5.1 CUDA:0 (Quadro P4000, 8192MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,826,706 parameters, 0 gradients, 41.7 GFLOPs
WARNING  Dataset 'split=val' not found, using 'split=test' instead.
train: C:\Users\kanan\Desktop\Work_at_IGB\Code\datasets\train... found 50442 images in 50 classes  
val: None...
test: C:\Users\kanan\Desktop\Work_at_IGB\Code\datasets\test... found 12632 images in 50 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 198/198 [00:24<00:00,  8.17it/s]


                   all      0.986          1
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train12
